In [54]:
from pymongo import MongoClient, errors

# MongoDB Atlas
str_pass = "url"
client = MongoClient(str_pass)
db = client['http_request_meta_data']
meta_collection = db['page_meta_data']

In [56]:
from functools import lru_cache, wraps
import time, logging

logger = logging.getLogger(__name__)
logging.basicConfig(level = logging.ERROR)

def retry(max_attempts = 3, delay = 1):
    """실패 시 retry 하는 decorator 입니다."""
    def decorator(func : any):
        @wraps(func)
        def wrapper(*args, **kwargs):
            attempts = 0
            while attempts < max_attempts:
                try:
                    return func(*args, **kwargs)
                except errors.PyMongoError as e:
                    logger.error(f"Error: {e}, Retrying... ({attempts+1}/{max_attempts})")
                    attempts += 1
                    time.sleep(delay)    
            raise Exception(f"Failed after {max_attempts} attempts")
        return wrapper
    return decorator

def timing(func):
    """타이머 데코레이터 입니다."""
    @wraps(func)
    def wrapper(*args, **kwargs):
        start_time = time.time()
        result = func(*args, **kwargs)
        end_time = time.time()
        print(f"{func.__name__} took {end_time - start_time:.4f} seconds")
        return result
    return wrapper

def validate_inputs(validation_func):
    """입력 값에 대한 유효성 검사를 하는 데코레이터입니다."""
    def decorator(func):
        @wraps(func)
        def wrapper(*args, **kwargs):
            if not validation_func(*args, **kwargs):
                raise ValueError("Invalid input values")
            return func(*args, **kwargs)
        return wrapper
    return decorator

In [57]:
@retry(max_attempts = 3, delay = 2)
@lru_cache(maxsize = 128)
def extract_mongodb_meta(collection):
    li = []
    for i in collection.find():
        li.append(i)
    return li

ab = extract_mongodb_meta(meta_collection)

ab

[{'_id': ObjectId('6656b0b184b94d6462d0cd0f'),
  'request': {'method': 'GET',
   'url': 'https://www.coupang.com/np/search?q=%EB%83%89%EC%9E%A5%EA%B3%A0&channel=recent&component=&eventCategory=SRP&trcid=&traid=&sorter=scoreDesc&minPrice=&maxPrice=&priceRange=&filterType=&listSize=36&filter=&isPriceRange=false&brand=&offerCondition=&rating=0&page=3&rocketAll=false&searchIndexingToken=1=9&backgroundColor=',
   'domain': 'www.coupang.com/np/search?q=%EB%83%89%EC%9E%A5%EA%B3%A0&channel=recent&component=&eventCategory=SRP&trcid=&traid=&sorter=scoreDesc&minPrice=&maxPrice=&priceRange=&filterType=&listSize=36&filter=&isPriceRange=false&brand=&offerCondition=&rating=0&page=3&rocketAll=false&searchIndexingToken=1=9&backgroundColor=',
   'header': {'accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.7',
    'accept-language': 'ko-KR,ko;q=0.9,en-US;q=0.8,en;q=0.7',
    'referer': 'https://www.google.com',

In [60]:
def is_positive(x):
    return x > 0

@validate_inputs(is_positive)
def process_data(x):
    return x ** 2

process_data(3)

9